In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service  # importa o serviço, quem diz pro Selenium onde está o chrome driver
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Faz a instalação do Chrome webdriver manager e passa o local para o serviço
servico = Service (ChromeDriverManager().install()) 
# Cria o navegador
navegador = webdriver.Chrome(service=servico)

In [28]:
dic_dfs = {}
df_produtos  = pd.read_excel("Buscas.xlsx")

# Iterando linha por linha
for index, row in df_produtos.iterrows():
    navegador.get("https://www.google.com/")
    nome_produto = row['Nome']
    nome_produto = nome_produto.lower()
    termos_banidos= row['Termos banidos']
    termos_banidos = termos_banidos.lower()
    preco_min = row['Preço mínimo']
    preco_max = row['Preço máximo']
    preco_min = float(preco_min)
    preco_max = float(preco_max)

    navegador.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(nome_produto,Keys.ENTER)

    
    # Espera 20 s para usuário passar pelo captcha
    esperar = WebDriverWait(navegador, 20).until(
        EC.presence_of_element_located((By.ID, 'logo')) 
    )
    
    el_aba_shopping = navegador.find_element(By.XPATH,'//*[@id="cnt"]/div[4]/div/div[1]/div/div/div[1]/div[2]/a').click()

    l_descricoes = []
    l_precos = []
    l_links =[]

    lista_anuncios = navegador.find_elements(By.CLASS_NAME,'mnr-c')

    for el_anuncio in lista_anuncios:

        descricao = el_anuncio.find_element(By.CLASS_NAME,'pymv4e')
        preco = el_anuncio.find_element(By.CLASS_NAME,'e10twf')
        link = el_anuncio.find_element(By.CLASS_NAME,'plantl').get_attribute('href')  
        
        contem_todos_termos = True 
        contem_termos_banidos = False

        #print('termo busca:',nome_produto.split())    
        #print('descrição:',descricao.text.lower())
        
        # Analisa se o produto contém TODOS os termos de busca
        for termo in nome_produto.split():
            if termo not in descricao.text.lower():
                contem_todos_termos = False
                break

        # Analisa se o produto não contém termos banidos
        for termo_banido in termos_banidos.split():
            if termo_banido in descricao.text.lower():
                contem_termos_banidos = True
                break
        #print('contem_todos_termos', contem_todos_termos)
        #print('contem_termos_banidos',contem_termos_banidos)

        if  contem_todos_termos  & (not contem_termos_banidos):
            
            preco = preco.text
            preco =preco.split()[1]
            # formata o número para padrão americano
            preco = float(preco.replace('.', '').replace(',', '.'))
            # Está entre o limite mínimo e máximo
            if (preco_min <= preco <= preco_max):
                l_descricoes.append(descricao.text)
                l_precos.append(preco)
                l_links.append(link)

    df_google = pd.DataFrame({
        'Descrição': l_descricoes,
        'Preço': l_precos,
        'Links': l_links
    })

    #df_google.to_excel("Pesquisa_Google_Shopping.xlsx")    
    l_descricoes = []
    l_precos = []
    l_links =[]

    navegador.get("https://www.buscape.com.br/")
    navegador.find_element(By.CLASS_NAME, 'AutoCompleteStyle_input__WAC2Y').send_keys(nome_produto,Keys.ENTER)
    lista_anuncios = navegador.find_elements(By.CLASS_NAME,"ProductCard_ProductCard_Inner__gapsh")
    for el_anuncio in lista_anuncios:
        link = el_anuncio.get_attribute('href')
        #print(link)
        # Localiza o elemento <h2> com o texto do produto
        descricao_el = el_anuncio.find_element(By.CSS_SELECTOR, 'h2[data-testid="product-card::name"]')
        # Extrai o texto
        descricao = descricao_el.text.lower()
        #print(descricao)

        # Localiza o elemento <p> com o preço do produto
        preco_el = el_anuncio.find_element(By.CSS_SELECTOR, 'p[data-testid="product-card::price"]')
        preco = preco_el.text
        # formata o número para padrão americano
        preco = preco.replace('.', '').replace(',', '.').replace('R$','').replace(' ','')
        preco = float(preco)


        contem_todos_termos = True 
        contem_termos_banidos = False
        
        # Analisa se o produto contém TODOS os termos de busca
        for termo in nome_produto.split():
            if termo not in descricao:
                contem_todos_termos = False
                break

        # Analisa se o produto não contém termos banidos
        for termo_banido in termos_banidos.split():
            if termo_banido in descricao:
                contem_termos_banidos = True
                break

        if  contem_todos_termos  & (not contem_termos_banidos):
            
            # Está entre o limite mínimo e máximo
            if (preco_min <= preco <= preco_max):
                l_descricoes.append(descricao)
                l_precos.append(preco)
                l_links.append(link)

    df_buscape = pd.DataFrame({
        'Descrição': l_descricoes,
        'Preço': l_precos,
        'Links': l_links
    })

    df_final = pd.concat([df_google, df_buscape], ignore_index=True)
    
    dic_dfs[nome_produto] = df_final

In [29]:
with pd.ExcelWriter('Resultado_Pesquisa.xlsx') as writer:
    for nome_produto, df in dic_dfs.items():
        # Trunca o nome da aba para 31 caracteres e remove caracteres inválidos
        aba_nome = nome_produto[:31].replace('/', '-').replace('\\', '-')
        df.to_excel(writer, sheet_name=aba_nome, index=False)
